# Testing with Structured Output

Making your output structured is always a good idea. It forces the LLM to generate all the required fields in the output, so it won't "forget" about important things.

And in addition to this and other benefits, testing your agent becomes much simpler.

Let's make our agent output more structured and predictable.

# Defining the Output Structure

Let's use this structure:

In [ ]:
from pydantic import BaseModel

class Reference(BaseModel):
    title: str
    filename: str

class Section(BaseModel):
    heading: str
    content: str
    references: list[Reference]

class SearchResultArticle(BaseModel):
    title: str
    sections: list[Section]
    references: list[Reference]

# Updating Agent Instructions

Since we don't need the full URL anymore (we can build it ourselves), we no longer need to ask the LLM to do it.

Let's tweak the prompt slightly.

In [ ]:
search_instructions = """
You are a helpful assistant that answers questions by searching
the documentation.

Requirements:

- For every user query, you must perform at least 3 separate searches
    to gather enough context and verify accuracy.  
- Each search should use a different angle, phrasing, or keyword
    variation of the user's query. 
- After performing all searches, write a concise, accurate answer
    that synthesizes the findings.  
- For each section, include references listing all the sources
    you used to write that section.
"""

# Configuring Structured Output

Don't forget to add output_type to the agent:

In [ ]:
def create_agent():
    tools = search_tools.prepare_search_tools()

    return Agent(
        name="search",
        instructions=search_instructions,
        tools=[tools.search],
        model="openai:gpt-4o-mini",
        output_type=SearchResultArticle,
    )


# Output Formatting

We can add this method for printing formatted articles:

In [ ]:
def format_article(
        self,
        base_url: str = "https://github.com/evidentlyai/docs/blob/main"
) -> str:
    output = [f"# {self.title}\n"]

    for section in self.sections:
        output.append(f"## {section.heading}\n")
        output.append(f"{section.content}\n")

        if section.references:
            output.append("### References\n")
            for ref in section.references:
                output.append(f"- {ref.title} ({base_url}/{ref.filename})\n")

        output.append("\n")

    if self.references:
        output.append("## All References\n")
        for ref in self.references:
            output.append(f"- {ref.title} ({ref.filename})\n")

    return "\n".join(output)


# Enhanced Testing with Structured Output

Now in tests we can access the fields of the new object directly:

In [ ]:
def test_agent_code():
    result = run_agent_sync("How do I implement LLM as a Judge eval?")

    tool_calls = get_tool_calls(result)
    assert len(tool_calls) >= 3, "Less than 3 tool calls found"

    article = result.output
    print(article.format_article())

    assert len(article.sections) > 0, "No sections found"
    assert len(article.references) > 0, "No references found"

    code_found = False

    for section in article.sections:
        if "```python" in section.content:
            code_found = True

        assert len(section.references) > 0, f"No references found in section {section.heading}"

    assert code_found, "No code block found in any section"


# Additional Test Ideas

We can add more checks.

For example, we can check that files from references actually exist in the documentation.

When building a deep research agent, we recorded the search queries in output. But we had a problem that these search queries didn't match the actual tool calls. With testing it's easy to catch.



# Test-Driven Development Approach

We can continue adding more tests and more scenarios with different prompts.

The key principle is: when you discover that something doesn't work the way it's intended, you write a test that reproduces this behavior. Then you fix the issue, and finally you run all tests to make sure you don't break anything else.

This approach ensures your agent remains reliable and predictable as you make improvements and add new features.